In [4]:
%load_ext autoreload
%autoreload 2

<div style="font-family: Vazirmatn; font-size:30px; direction: rtl; display: flex; align-items:center; justify-content:center; align-text:center; background-color: #d6336c; padding: 20px 40px;border: 1px solid black; border-radius: 5px">
مدلسازی و حل مسئله
</div>

In [5]:
# ---------------------------------------------------------------------------- #
#                              Import the modules                              #
# ---------------------------------------------------------------------------- #
from pyomo.environ import *

# ---------------------------------------------------------------------------- #
#                             Initialize the MODEL                             #
# ---------------------------------------------------------------------------- #

model = ConcreteModel()
beans = [1, 2, 3, 4]
brands = ['A', 'B', 'C']

# ---------------------------------------------------------------------------- #
#                               Define the PARAMS                              #
# ---------------------------------------------------------------------------- #
cost = {1: 0.23, 2: 0.20, 3: 0.15, 4: 0.10}
availability = {1: 4000, 2: 6000, 3: 2500, 4: 8000}
production = {'A': 5000, 'B': 10000, 'C': 3000}

# ---------------------------------------------------------------------------- #
#                             Define the VARIABLES                             #
# ---------------------------------------------------------------------------- #
model.x = Var(beans, brands, domain=NonNegativeReals)

# ---------------------------------------------------------------------------- #
#                         Define the OBJECTIVE FUNCTION                        #
# ---------------------------------------------------------------------------- #
model.objective = Objective(
    expr=sum(cost[i] * model.x[i, j] for i in beans for j in brands),
    sense=minimize
)

# ---------------------------------------------------------------------------- #
#                                  Constraints                                 #
# ---------------------------------------------------------------------------- #

# ----------------- 1. Production requirements for each brand ---------------- #
model.production_constraints = ConstraintList()
for j in brands:
    model.production_constraints.add(
        sum(model.x[i, j] for i in beans) >= production[j]
    )

# ----------------- 2. Availability constraints for each bean ---------------- #
model.availability_constraints = ConstraintList()
for i in beans:
    model.availability_constraints.add(
        sum(model.x[i, j] for j in brands) <= availability[i]
    )

model.zero_constrain1 = Constraint(expr=model.x[1,'A']==0)
model.zero_constrain2 = Constraint(expr=model.x[3,'B']==0)
model.zero_constrain3 = Constraint(expr=model.x[4,'C']==0)

# -------------------------- 3. Brand specifications ------------------------- #
# Brand A
model.brand_A_constraints = ConstraintList()
model.brand_A_constraints.add(
    model.x[2, 'A'] >= 0.10 * sum(model.x[i, 'A'] for i in beans)
)
model.brand_A_constraints.add(
    model.x[3, 'A'] >= 0.40 * sum(model.x[i, 'A'] for i in beans)
)
model.brand_A_constraints.add(
    model.x[3, 'A'] <= 0.70 * sum(model.x[i, 'A'] for i in beans)
)
model.brand_A_constraints.add(
    model.x[4, 'A'] <= 0.80 * sum(model.x[i, 'A'] for i in beans)
)

# Brand B
model.brand_B_constraints = ConstraintList()
model.brand_B_constraints.add(
    model.x[1, 'B'] >= 0.20 * sum(model.x[i, 'B'] for i in beans)
)
model.brand_B_constraints.add(
    model.x[1, 'B'] <= 0.40 * sum(model.x[i, 'B'] for i in beans)
)
model.brand_B_constraints.add(
    model.x[2, 'B'] >= 0.30 * sum(model.x[i, 'B'] for i in beans)
)
model.brand_B_constraints.add(
    model.x[2, 'B'] <= 0.70 * sum(model.x[i, 'B'] for i in beans)
)
model.brand_B_constraints.add(
    model.x[4, 'B'] >= 0.10 * sum(model.x[i, 'B'] for i in beans)
)

# Brand C
model.brand_C_constraints = ConstraintList()
model.brand_C_constraints.add(
    model.x[1, 'C'] >= 0.40 * sum(model.x[i, 'C'] for i in beans)
)
model.brand_C_constraints.add(
    model.x[2, 'C'] <= 0.60 * sum(model.x[i, 'C'] for i in beans)
)
model.brand_C_constraints.add(
    model.x[3, 'C'] <= 0.10 * sum(model.x[i, 'C'] for i in beans)
)

# ---------------------------------------------------------------------------- #
#                                Solve the model                               #
# ---------------------------------------------------------------------------- #
solver = SolverFactory('cplex')
solver.solve(model)

print(f'objective function : {value(model.objective)}')

for x in beans:
    for y in brands:
        print(f'value of {model.x[x,y]}={value(model.x[x,y])}')

objective function : 2831.0
value of x[1,A]=0.0
value of x[1,B]=2000.0
value of x[1,C]=1200.0
value of x[2,A]=500.0
value of x[2,B]=3000.0
value of x[2,C]=1500.0
value of x[3,A]=2000.0
value of x[3,B]=0.0
value of x[3,C]=300.0
value of x[4,A]=2500.0
value of x[4,B]=5000.0
value of x[4,C]=0.0
